<div class="alert alert-block alert-success">
<center><b>AIM404 : Contextual Bandits with Amazon SageMaker RL</b></center>
</div>

We demonstrate how you can manage your own contextual multi-armed bandit workflow on SageMaker using the built-in [Vowpal Wabbit (VW)](https://github.com/VowpalWabbit/vowpal_wabbit) container to train and deploy contextual bandit models. We show how to train these models that interact with a live environment (using a simulated client application) and continuously update the model with efficient exploration.

### Why Contextual Bandits?

Wherever we look to personalize content for a user (content layout, ads, search, product recommendations, etc.), contextual bandits come in handy. Traditional personalization methods collect a training dataset, build a model and deploy it for generating recommendations. However, the training algorithm does not inform us on how to collect this dataset, especially in a production system where generating poor recommendations lead to loss of revenue. Contextual bandit algorithms help us collect this data in a strategic manner by trading off between exploiting known information and exploring recommendations which may yield higher benefits. The collected data is used to update the personalization model in an online manner. Therefore, contextual bandits help us train a personalization model while minimizing the impact of poor recommendations.

### What does this notebook contain?

To implement the exploration-exploitation strategy, we need an iterative training and deployment system that: (1) recommends an action using the contextual bandit model based on user context, (2) captures the implicit feedback over time and (3) continuously trains the model with incremental interaction data. In this notebook, we show how to setup the infrastructure needed for such an iterative learning system. While the example demonstrates a bandits application, these continual learning systems are useful more generally in dynamic scenarios where models need to be continually updated to capture the recent trends in the data (e.g. tracking fraud behaviors based on detection mechanisms or tracking user interests over time). 

In a typical supervised learning setup, the model is trained with a SageMaker training job and it is hosted behind a SageMaker hosting endpoint. The client application calls the endpoint for inference and receives a response. In bandits, the client application also sends the reward (a score assigned to each recommendation generated by the model) back for subsequent model training. These rewards will be part of the dataset for the subsequent model training. 

<p align="center">
  <img src="images/AIM404-workflow.png">
</p>

The contextual bandit training workflow is controlled by an experiment manager provided with this example. The client application (say a recommender system application) pings the SageMaker hosting endpoint that is serving the bandits model. The application sends the state (user features) as input and receives an action (recommendation) as a response. The client application sends the recommended action to the user and stores the received reward in S3. The SageMaker hosted endpoint also stores inference data (state and action) in S3. The experiment manager joins the inference data with rewards as they become available. The joined data is used to update the model with a SageMaker training job. The updated model is evaluated offline and deployed to SageMaker hosting endpoint if the model evaluation score improves upon prior models. 

Below is an overview of the subsequent cells in the notebook: 
* <a href='#configuration'>Configuration:</a> this includes details related to SageMaker and other AWS resources needed for the bandits application. 
* <a href='#iam-role-setup'>IAM role setup:</a> this creates appropriate execution role and shows how to add more permissions to the role, needed for specific AWS resources.
* <a href='#client-application'>Client application (Environment):</a> this shows the simulated client application.
* <a href='#step-by-step'>Step-by-step bandits model development:</a> 
 1. Model Initialization (random or warm-start) and evaluation
 2. Deploy the First Model 
 3. Initialize the Client Application : Start Inference
 4. Reward Ingestion 
 5. Model Re-training and Re-deployment 
* <a href='#end-to-end'>Bandits model deployment with the end-to-end loop</a>
* <a href='#clean-up'>Cleanup</a>

### Conventions - Markdown colors convention
In all notebooks, there is color convention on the cells:
<div class="alert alert-block alert-warning"><b>Yellow cells</b>: These cells give you context on the steps or work expected within a notebook</div> 
<div class="alert alert-block alert-info"><b>Blue cells</b>: These cells contain actions that you will run or execute</div>
<div class="alert alert-block alert-success"><b>Green cells</b>: These cells give a summary report of what you have accomplished so far in a notebook</div>
<div class="alert alert-block alert-danger""><b>Red cells</b>: These cells give an alert or a warning for an action for participants to pay attention</div>

<a id='configuration'></a>
## Configuration

To facilitate experimentation, we provide a `local_mode` that runs the contextual bandit example using the SageMaker Notebook instance itself instead of SageMaker training and hosting instances. The workflow remains the same in `local_mode`, but runs much faster for small datasets. Hence, it is a useful tool for experimentation and debugging. However, it will not scale to production use cases with high throughput and large datasets. 

In `local_mode`, the training, evaluation and hosting is done with the SageMaker VW docker container. The join is not handled by SageMaker, and is done inside the client application. The rest of the textual explanation assumes that the notebook is run in SageMaker mode.

In [ ]:
import yaml
import sys
import numpy as np
import time
import sagemaker
import pprint
import pandas as pd
sys.path.append('common')
sys.path.append('common/sagemaker_rl')
from misc import get_execution_role
from markdown_helper import *
from IPython.display import Markdown
from IPython.core.display import Image, display, HTML

<div class="alert alert-block alert-danger"">
<b>IMPORTANT :</b> You need to run the notebook with Sagemaker only, so you need to configure the following value in config.yaml file 
<ul>
                                            <li>local_mode: <b>false</b></li>
                                            <li>soft_deployment: <b>false</b></li>
</ul>
</div>

In [ ]:
!pygmentize 'config.yaml'
config_file = 'config.yaml'
with open(config_file, 'r') as yaml_file:
    config = yaml.load(yaml_file)

#### Vowpal Wabbit 

Let's do a deep dive on the Vowpal Wabbit (VW) configuration!

> A container for Vowpal Wabbit is already provisionned by AWS and available here : 
`462105765813.dkr.ecr.<region>.amazonaws.com/sagemaker-rl-vw-container:vw-8.7.0-cpu`.

This container is built and maintained by AWS Teams but you can access the DockerFile!




<div class="alert alert-block alert-info">
§ Open the <a href="https://github.com/aws/sagemaker-rl-container/blob/master/vw/docker/8.7.0/Dockerfile">Vowpal Wabbit Dockerfile</a>
</div>

VW supports multiple **Contextual Bandit algorithms**:

- Explore-first
- **Epsilon-greedy** - egreedy
- **Bagging Explorer** - bag
- **Online Cover** - cover


> Please make sure that the `num_arms` parameter in the config is equal to the number of actions in the client application (which is defined in the cell below).

<a id='iam-role-setup'></a>
## IAM role setup
Either get the execution role when running from a SageMaker notebook `role = sagemaker.get_execution_role()` or, when running from local machine, use `utils` method `role = get_execution_role('role_name')` to create an execution role.

<div class="alert alert-block alert-warning">
    Retrieve Sagemaker IAM role arn
</div/>

In [ ]:
try:
    sagemaker_role = sagemaker.get_execution_role()
except:
    sagemaker_role = get_execution_role('sagemaker')

print("Using Sagemaker IAM role arn: \n{}".format(sagemaker_role))

<a id='client-application'></a>
## Client application (Environment) 

#### [Statlog (Shuttle) Data Set](https://archive.ics.uci.edu/ml/datasets/Statlog+(Shuttle))

<div class="alert alert-block alert-warning">
    It contains <b>nine integer attributes (or features)</b> related to indicators during a space shuttle flight, and the goal is to predict one of seven states of the radiator subsystem of the shuttle.
    
   It can be seen as a multi-class classification problem with 9 features and 7 classes. In the classification problem, the algorithm receives features and correct label per datapoint.
    
   Here, we convert this multi-class classifcation problem to a <b>bandit problem.</b>
    
   The algorithm picks one of the label (arm) options given the features (context). If this matches the class in the original data point, a reward of one is assigned. If not, a reward of zero is assigned.
</div/>

In [ ]:
display(Image('images/AIM404-classification-bandits.png'))

The client application simulates a live environment that uses **the SageMaker bandits model to serve recommendations to users**. The logic of reward generation resides in the client application.

The workflow of the client application is as follows:
- The client application picks a context at random, which is sent to the SageMaker endpoint for retrieving an action.
- SageMaker endpoint returns an action, associated probability and `event_id`.
- Since this simulator was generated from the Statlog dataset, we know the true class for that context. 
- The application reports the reward to the experiment manager using S3, along with the corresponding `event_id`.

`event_id` is a unique identifier for each interaction. It is used to join inference data `<state, action, action probability>` with the rewards. 

In a later cell of this notebook, where there exists a hosted endpoint, we illustrate how the client application interacts with the endpoint and gets the recommended action.

The shuttle data set is stored under **sim_app/shuttle.trn**

They are actually 2 scripts that will help us simulate our application : 
* sim_app_utils.py
* statlog_sim_app.py

In [ ]:
sys.path.append('sim_app')
from statlog_sim_app import StatlogSimApp

In [ ]:
!pygmentize sim_app/sim_app_utils.py

In [ ]:
!pygmentize sim_app/statlog_sim_app.py

<a id='step-by-step'></a>
## Step-by-step bandits model development

`ExperimentManager` is the top level class for all the Bandits/RL and continual learning workflows. Similar to the estimators in the [Sagemaker Python SDK](https://github.com/aws/sagemaker-python-sdk), `ExperimentManager` contains methods for training, deployment and evaluation. It keeps track of the job status and reflects current progress in the workflow.

Start the application using the `ExperimentManager` class. Information about the class can be found here : **/common/sagemaker_rl/orchestrator/workflow/manager/experiment_manager.py**

<div class="alert alert-block alert-warning">
    <b>Experiment Manager Methods </b>

***Training***

- initialize_first_model( ) : Initializes the first Model training for an Experiment
- evaluate_model( ) : Start an evaluation job to evaluate a model
- train_next_model( ) : Train a new model given the training data and a pretrained model

***Inference***

- deploy_model( ) : Deploy a new model by creating a new hosting endpoint or update the model hosted by an existing endpoint
- predictor( ) : Create a predictor
- get_eval_score( ) :  Return evaluation score given model id and evaluation data path

***Data Aggregation***

- ingest_rewards( ) : Upload rewards data in a rewards buffer to S3 bucket
- ingest_joined_data( ) : Upload joined data in joined data buffer to S3 bucket
- join( ) : Start a joining job given rewards data path and observation data time window

***Logging***

- get_cloudwatch_dashboard_details( ) : Get detail from cloudwatch 

***Clean-up***

- clean_resource( ) : Clean up resource of the given experiment, including hosting endpoint and firehose stream
- clean_table_records( ) : Clean up all related records of a given experiment
</div/>

In [ ]:
from orchestrator.workflow.manager.experiment_manager import ExperimentManager

 The initialization below will set up an AWS CloudFormation stack of additional resources. 

<div class="alert alert-block alert-danger"">
<b>IMPORTANT:</b> Enter the name of your Experiment here : 
<ul>
  <li>model_id length cannot exceed 63 characters under SM mode.</li>
  <li>evaluation job name will include timestamp in addition to train job name so, make experimend_id as short as possible</li>
</ul>
</div>

In [ ]:
experiment_name = "AIM404-sbs" #YOUR EXPERIMENT NAME HERE - can be AIM404-1
bandits_experiment = ExperimentManager(config, experiment_id=experiment_name)

<div class="alert alert-block alert-info">
§ Open the <a href="https://us-east-1.console.aws.amazon.com/cloudformation/home?force&region=us-east-1">Cloudformation console</a> to understand which services are being setup
</div>

### 1. Model Initialization and evaluation

To start a new experiment, we need to initialize the first model. In the case where historical data is available and is in the format of `<state, action, action probability, reward>`, we can warm start by learning the policy offline. Otherwise, we can initiate a random policy.

**Warm start the policy**

We showcase the warm start by generating a batch of randomly selected samples with size `batch_size`. Then we split it into a training set and an evaluation set using the parameter `ratio`.

From sim_app/sim_app_utils.py, we use a tool that simulate an application

In [ ]:
display(Image('images/AIM404-model-initialization.png'))

In [ ]:
from sim_app_utils import *

batch_size = 100
warm_start_data_buffer = prepare_statlog_warm_start_data(data_file='sim_app/shuttle.trn', batch_size=batch_size)

# upload to s3
bandits_experiment.ingest_joined_data(warm_start_data_buffer,ratio=0.8)

With prepare_statlog_warm_start_data() method, it randomly picks an action and generate the reward associated to the action picked!

<div class="alert alert-block alert-warning">
    <b>Check the status of bandits_experiment object</b>

It is composed of the following elements : 
- experiment_id
- evaluation_workflow_metadata
- hosting_workshow_metadata
- joining_workflow_metadata
- training_workflow_metadata
</div/>

In [ ]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(bandits_experiment._jsonify())

In [ ]:
bandits_experiment.initialize_first_model(input_data_s3_prefix=bandits_experiment.last_joined_job_train_data) 

<div class="alert alert-block alert-info">
§ Open the <a href="https://us-east-1.console.aws.amazon.com/sagemaker/home?force&region=us-east-1#/jobs?f0=kvm%253AName%253AAIM404-sbs">Sagemaker console</a> to understand what is happening behind the scene
</div>

<div class="alert alert-block alert-warning">

The Experiment manager relies on [RLEstimator](https://sagemaker.readthedocs.io/en/stable/sagemaker.rl.html) from Sagemaker SDK to do the training and deployment.

The custom code for training is located in **train-vw.py** located in **src**
</div/>


In the experiment workflow, we have trained a specific model

In [ ]:
pp.pprint(bandits_experiment._jsonify())

**Evaluate current model against historical model**

After every training cycle, we evaluate if the newly trained model is better than the one currently deployed. Using the evaluation dataset, we evaluate how the new model would perform compared to the model that is currently deployed. SageMaker RL supports offline evaluation by performing counterfactual analysis (CFA). By default, we apply [doubly robust (DR) estimation](https://arxiv.org/pdf/1103.4601.pdf) method. The bandit policy tries to **minimize the cost (1-reward)** value in this case, so a smaller evaluation score indicates better policy performance.

In [ ]:
display(Image('images/AIM404-model-initialization-evaluate.png'))

In [ ]:
# evaluate the current model by launching a training job on Sagemaker
bandits_experiment.evaluate_model(
    input_data_s3_prefix=bandits_experiment.last_joined_job_eval_data,
    evaluate_model_id=bandits_experiment.last_trained_model_id)

<div class="alert alert-block alert-info">
§ Open the <a href="https://us-east-1.console.aws.amazon.com/sagemaker/home?force&region=us-east-1#/jobs?f0=kvm%253AName%253AAIM404-sbs">Sagemaker console</a> to understand what is happening during evaluation
</div>

<div class="alert alert-block alert-warning">

The Experiment manager uses the same estimator for evalution and training ==> [RLEstimator](https://sagemaker.readthedocs.io/en/stable/sagemaker.rl.html) 

The difference here is the custom code used as entry point for the container. It is now **eval-cfa-vw.py** in **src**

Once again, the evaluation of our model should be between 0 and 1. The smaller, the better
</div/>

In [ ]:
# now get the evaluation score
eval_score_last_trained_model = bandits_experiment.get_eval_score(
    evaluate_model_id=bandits_experiment.last_trained_model_id,
    eval_data_path=bandits_experiment.last_joined_job_eval_data
)

<div class="alert alert-block alert-warning">
Let's evaluate the model performance on our historical data set (here the warm start). First, we will download the file statlog_warm_start.data
</div/>

Download last joined data used for evaluation under **statlog_warm_start.data** file

In [ ]:
download_historical_data_from_s3(data_s3_prefix=bandits_experiment.last_joined_job_eval_data)

Check what is inside

In [ ]:
pd.read_csv("./statlog_warm_start.data", sep=',')

In [ ]:
# get baseline performance from the historical (warm start) data (cost = 1-mean(reward))
baseline_score = evaluate_historical_data(data_file='statlog_warm_start.data')
baseline_score

Now we can compare the value from **eval_score_last_trained_model** with the **baseline_score**. Can you see an improvement? :)

In [ ]:
# Check the model_id of the last model trained.
bandits_experiment.last_trained_model_id

In [ ]:
pp.pprint(bandits_experiment._jsonify())

<div class="alert alert-block alert-success">
    So far, we uploaded data on S3, trained a model on this dataset and evaluated it.</div>

### 2. Deploy the First Model

Once training and evaluation is done, we can deploy the model.

In [ ]:
display(Image('images/AIM404-model-initialization-deploy.png'))

In [ ]:
bandits_experiment.deploy_model(model_id=bandits_experiment.last_trained_model_id) 

<div class="alert alert-block alert-info">
§ Open the <a href="https://us-east-1.console.aws.amazon.com/sagemaker/home?force&region=us-east-1#/endpoints?f0=kvm%253AName%253AAIM404-sbs">Sagemaker Endpoint</a>
</div>

<div class="alert alert-block alert-warning">

SageMaker hosting endpoint saves all the inferences `<eventID, state, action, action probability>` to S3 using [Kinesis Firehose](https://aws.amazon.com/kinesis/data-firehose/)

This firehose stream is created during endopint deployment.


The Experiment manager creates the sagemaker endpoint by using the method [Model](https://sagemaker.readthedocs.io/en/stable/model.html) from Sagemaker SDK.
</div/>

<div class="alert alert-block alert-info">
§ Open the <a href="https://us-east-1.console.aws.amazon.com/firehose/home?region=us-east-1#/dashboard/list">Kinesis Firehose Console</a>
</div>

You can check the experiment state at any point by executing:

In [ ]:
pp.pprint(bandits_experiment._jsonify())

The model just trained appears in both `last_trained_model_id` and `last_hosted_model_id`.

<div class="alert alert-block alert-success">
    Now we deployed a Sagemaker Endpoint to interactively request it.</div>

### 3. Initialize the Client Application

Now that the last trained model is hosted, client application can send out the state, hit the endpoint, and receive the recommended action. There are 7 classes in the statlog data, corresponding to 7 actions respectively.

In [ ]:
display(Image('images/AIM404-inference.png'))

<div class="alert alert-block alert-warning">

Here we use the [Predictor](https://sagemaker.readthedocs.io/en/stable/predictors.html) class from Sagemaker Python SDK.
</div/>

In [ ]:
predictor = bandits_experiment.predictor

In [ ]:
sim_app = StatlogSimApp(predictor=predictor)

Make sure that `num_arms` specified in `config.yaml` is equal to the total unique actions in the simulation application.

In [ ]:
assert sim_app.num_actions == bandits_experiment.config["algor"]["algorithms_parameters"]["num_arms"]

In [ ]:
user_id, user_context = sim_app.choose_random_user()
action, event_id, model_id, action_prob, sample_prob = predictor.get_action(obs=user_context)

# Check prediction response by uncommenting the lines below
print('Selected action: {}, event ID: {}, model ID: {}, probability: {}'.format(action, event_id, model_id, action_prob))

<div class="alert alert-block alert-success">
    Here we simulated a client request to the sagemaker endpoint with the get_action() method that sends context to the sagemaker endpoint</div>

### 4. Reward Ingestion

Client application generates a reward after receiving the recommended action and stores the tuple `<eventID, reward>` in S3. In this case, reward is 1 if predicted action is the true class, and 0 otherwise. . The experiment manager joins the reward with state, action and action probability using [Amazon Athena](https://aws.amazon.com/athena/). 

In [ ]:
display(Image('images/AIM404-reward.png'))

In [ ]:
local_mode = bandits_experiment.local_mode
batch_size = 500 # collect 500 data instances
print("Collecting batch of experience data...")

# Generate experiences and log them
for i in range(batch_size):
    user_id, user_context = sim_app.choose_random_user()
    action, event_id, model_id, action_prob, sample_prob = predictor.get_action(obs=user_context.tolist())
    reward = sim_app.get_reward(user_id, action, event_id, model_id, action_prob, sample_prob, local_mode)

#### Aggregate Inference data with rewards data

<div class="alert alert-block alert-warning">

Once data from inference and reward phase are on S3, we need to start aggregate this data to the following format : `<eventID, state, action, action probability, reward>`.

This step is done with [Amazon Athena](https://aws.amazon.com/athena/)
</div/>

In [ ]:
display(Image('images/AIM404-join.png'))

In [ ]:
# Join (observation, action) with rewards (can be delayed) and upload the data to S3
if local_mode:
    bandits_experiment.ingest_joined_data(sim_app.joined_data_buffer)
else:
    print("Waiting for firehose to flush data to s3...")
    time.sleep(60) # Wait for firehose to flush data to S3 - actually the VW container image take as a venv the firehose stream name
    rewards_s3_prefix = bandits_experiment.ingest_rewards(sim_app.rewards_buffer)
    print(rewards_s3_prefix)
    bandits_experiment.join(rewards_s3_prefix)
    
sim_app.clear_buffer()

<div class="alert alert-block alert-info">
§ Open the <a href="https://us-east-1.console.aws.amazon.com/athena/home?force&region=us-east-1#query">Athena console</a> and check history console to look at the different requests.
</div>

In [ ]:
bandits_experiment.last_joined_job_train_data

In [ ]:
# Check the workflow to see if join job has completed successfully
pp.pprint(bandits_experiment._jsonify())

<div class="alert alert-block alert-success">
    Here we ingested data from the simulated client and aggregated it with the inference data</div>

### 5. Model Re-training and Re-deployment

Now we can train a new model with newly collected experiences, and host the resulting model.

In [ ]:
display(Image('images/AIM404-model-retraining.png'))

In [ ]:
bandits_experiment.train_next_model(input_data_s3_prefix=bandits_experiment.last_joined_job_train_data)

<div class="alert alert-block alert-info">
§ Open the <a href="https://us-east-1.console.aws.amazon.com/sagemaker/home?force&region=us-east-1#/jobs?f0=kvm%253AName%253AAIM404-sbs">Sagemaker Training Jobs console</a>
</div>

In [ ]:
bandits_experiment.last_trained_model_id

#### Blue/Green Deployment

Now we will update the model behind the same sagemaker endpoint. We can check how blue/green deployment works on Sagemaker by analysing the following elements : 
    - Sagemaker Endpoints
    - Sagemaker Endpoint Configurations
    - Sagemaker Models
    
To check no error is happening during the blue/green deployment phase, you will send inference requests to the endpoint through a loop. 

<div class="alert alert-block alert-danger"">
To do so, go to the following notebooks <a href="./Inference%20loop.ipynb"> Inference Loop </a>
</div>

Now you can start the deployment of the newly trained model and follow that everything is ok through Cloudwatch

<div class="alert alert-block alert-info">
§ Open the <a href="https://us-east-1.console.aws.amazon.com/sagemaker/home?force&region=us-east-1#/endpoints/AIM404-sbs">Sagemaker Endpoints console</a>
</div>

<div class="alert alert-block alert-danger"">
<ul>
<li>Clink on your Sagemaker Endpoint</li>
<li>Under Monitor, click <b>View Invocation metrics</b>. This will open cloudwatch console</li>
                                            <li>Select Metric Name <b>Invocations</b> for your endpoint</li>
<li>Go to Graphed metrics tab</li>
                                            <li>Set Statistic to sum, period to 10 seconds</li>
                                            <li>On the upper right, select custom 5 min and select stacked instead of line</li>                                            
</ul>                                       
</div>

In [ ]:
# deployment takes ~10 min if `local_mode` is False
bandits_experiment.deploy_model(model_id=bandits_experiment.last_trained_model_id)

<div class="alert alert-block alert-info">
§ Open the <a href="https://us-east-1.console.aws.amazon.com/sagemaker/home?force&region=us-east-1#/endpoints">Sagemaker Endpoints console</a>
</div>

In [ ]:
bandits_experiment.last_hosted_model_id

<div class="alert alert-block alert-success">
    In this step, we deployed the new model version using sagemaker Blue/Green deployment method</div>

<a id='end-to-end'></a>
## Bandits model deployment with the end-to-end loop

 <div class="alert alert-block alert-danger"">
Go to the End-to-End notebook for the end of the workshop <a href="./AIM404-End_2_end_loop.ipynb"> AIM404-End_2_end_loop </a>
</div>

<a id='clean-up'></a>
## Clean up

We have three DynamoDB tables (experiment, join, model) from the bandits application above (e.g. `experiment_id='bandits-exp-1'`). To better maintain them, we should remove the related records if the experiment has finished. Besides, having an endpoint running will incur costs. Therefore, we delete these components as part of the clean up process.

<div class="alert alert-block alert-warning">

Only execute the clean up cells below when you've finished the current experiment and want to deprecate everything associated with it. After the cleanup, the Cloudwatch metrics will not be populated anymore. 
</div/>

In [ ]:
bandits_experiment.clean_resource(experiment_id=bandits_experiment.experiment_id)

<div class="alert alert-block alert-success">
    The command <b>clean_resource( )</b> deletes : 
<ul>
    <li>kinesis firehose</li>
    <li>the external table created by Athena</li>
    <li>sagemaker endpoint</li>
    <li>sagemaker endpoint configuration</li>
    </ul>
</div>

In [ ]:
bandits_experiment.clean_table_records(experiment_id=bandits_experiment.experiment_id)

<div class="alert alert-block alert-success">
    The command <b>clean_table_records( )</b> clean items from dynamoDB tables
</div>